In [ ]:
from pyspark.sql import SparkSession

In [ ]:
# 버킷명 읽어오기
import os

bucket_name = os.environ.get('S3_BUCKET_NAME')

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages "org.apache.hadoop:hadoop-aws:3.3.4" pyspark-shell'

In [ ]:
from pyspark.sql import SparkSession

from konlpy.tag import Kkma
from konlpy.utils import pprint

from collections import Counter
import os

from wordcloud import WordCloud 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages "org.apache.hadoop:hadoop-aws:3.3.4" pyspark-shell'

spark = SparkSession.builder \
    .appName("S3JSONAnalysis") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .getOrCreate()

# konlpy.jvm.init_jvm(jvmpath=None, max_heap_size=2048)
kkma = Kkma(max_heap_size=1024 * 4)


In [ ]:
webtoon_nos = ['131385', '183559', '21815', '400739', '552960', '570503', '570506', '597447', '602910', '602916', '616239', '641253', '644112', '648419', '64997', '650305', '654774', '670145', '671421', '676695', '687915', '695796', '697533', '698918', '701535', '702608', '703630', '703839', '703844', '703846', '703847', '710747', '710751', '711422', '712362', '717481', '721433', '721948', '725586', '727188', '728128', '728750', '729036', '730425', '730607', '730656', '730657', '732021', '732036', '732988', '733074', '733280', '734574', '735078', '735661', '736277', '738487', '738694', '739115', '739411', '740132', '741449', '741825', '741891', '743139', '743838', '745654', '745876', '746534', '746833', '746857', '747269', '747271', '748535', '748536', '748831', '750184', '750558', '750826', '751168', '751993', '751999', '752414', '752532', '753304', '753478', '753839', '753842', '753975', '754876', '755744', '756137', '756140', '757904', '758150', '758439', '758587', '758660', '758662', '758665', '758666', '758667', '758675', '758677', '759567', '759940', '760001', '760002', '761461', '761599', '761722', '762035', '764129', '764623', '765156', '765158', '765804', '766563', '766648', '766966', '767908', '768469', '768473', '768474', '768534', '768536', '769193', '769568', '769658', '769662', '769663', '770068', '770523', '771018', '771065', '771718', '771912', '772243', '772725', '772734', '772764', '772853', '773473', '773476', '773793', '773796', '773797', '774039', '774040', '774044', '774302', '774358', '774451', '774703', '774831', '774864', '774866', '775140', '775141', '775334', '775631', '776092', '776255', '776256', '776542', '776601', '776668', '777515', '777767', '778228', '778284', '778322', '778325', '778580', '778656', '778707', '778963', '778967', '778968', '778991', '779354', '779632', '780063', '780137', '780170', '780191', '780253', '780404', '780409', '780414', '780845', '783050', '783052', '783053', '783117', '783520', '783523', '783524', '783532', '783536', '783590', '783595', '783599', '783814', '783861', '783862', '783863', '783864', '783865', '783877', '783888', '783971', '784140', '784248', '784417', '784506', '784507', '784518', '784580', '784582', '784989', '785251', '785253', '785260', '785701', '785703', '785727', '785749', '785855', '786082', '786262', '786973', '786978', '786979', '787465', '787500', '788167', '788170', '788976', '789029', '789591', '789651', '789652', '789682', '789966', '789979', '790239', '790245', '790404', '790409', '790416', '790713', '790840', '791062', '791126', '791140', '791205', '791256', '791258', '791616', '791629', '791632', '792120', '792125', '792139', '792277', '792280', '792317', '792372', '792651', '792654', '792694', '792780', '793067', '793113', '793130', '793274', '793275', '793283', '793374', '793388', '793410', '793539', '793553', '793696', '793853', '793887', '793944', '794102', '794105', '794155', '794156', '794224', '794383', '794417', '794421', '794423', '794458', '794644', '794671', '794742', '794743', '794939', '794942', '794944', '794965', '795041', '795262', '795297', '795330', '795529', '795537', '795540', '795542', '795643', '795649', '795658', '795929', '796059', '796062', '796075', '796152', '796242', '796247', '796251', '796252', '796302', '796324', '796368', '796466', '796492', '796827', '796894', '796903', '796980', '797105', '797115', '797155', '797184', '797221', '797253', '797258', '797259', '797413', '797417', '797442', '797443', '797468', '797582', '797731', '797760', '797819', '797844', '797932', '797937', '797943', '798101', '798172', '798173', '798182', '798277', '798293', '798303', '798326', '798331', '798622', '798664', '798917', '799016', '799148', '799165', '799213', '799220', '799226', '799250', '799267', '799467', '799478', '799503', '799509', '799524', '799805', '799837', '799838', '799868', '800006', '800034', '800046', '800101', '800291', '800304', '800312', '800331', '800333', '800390', '800392', '800400', '800504', '800506', '800585', '800618', '800726', '800770', '800778', '800788', '800796', '800828', '801035', '801038', '801277', '801324', '801428', '801455', '801475', '801505', '801515', '801516', '801517', '801555', '801580', '801589', '801593', '801698', '801699', '801710', '801711', '801736', '801773', '801809', '801820', '801827', '801992', '801998', '802033', '802039', '802070', '802079', '802117', '802272', '802278', '802359', '802378', '802389', '802551', '802578', '802682', '802733', '802751', '802819', '802833', '802835', '802840', '802849', '802854', '802865', '802872', '802906', '802913', '802940', '802986', '803010', '803119', '803122', '803123', '803222', '803371', '803480', '803518', '803530', '803649', '803767', '803792', '803794', '803891', '803909', '803934', '804051', '804055', '804145', '804157', '804160', '804329', '804333', '804469', '804653', '804782', '804783', '804832', '804862', '804868', '804871', '804989', '805156', '805193', '805203', '805323', '805334', '805356', '805368', '805405', '805437', '805481', '805658', '805671', '805691', '805702', '805737', '805743', '805893', '805907', '805974', '806189', '806265', '806304', '806449', '806505', '806776', '806888', '807019', '807029', '807159', '807164', '807178', '807206', '807306', '807308', '807314', '807356', '807393', '807397', '807406', '807537', '807550', '807582', '807600', '807634', '807655', '807745', '807775', '807776', '807777', '807826', '807829', '807831', '807858', '807992', '808018', '808074', '808198', '808201', '808265', '808272', '808286', '808388', '808389', '808439', '808454', '808473', '808482', '808508', '808533', '808677', '808711', '808738', '808760', '808776', '808804']

In [ ]:
for webtoon_no in webtoon_nos: 
    loaded_df = spark.read.json(f"complete_hangul/complete_hangul_output_{webtoon_no}.json")
    sentences = loaded_df\
                .select("sentence")\
                .rdd\
                .flatMap(lambda x: x)\
                .collect()
    
    comment_nouns = []
    for i, sentence in enumerate(sentences):    
        if not sentence:
            continue
        
        cp = True
        for x in sentence.split():
            if len(x) > 13:
                cp = False
                print(f"This is strange: {x}. cp to be False")
                break
        
        if not cp:
            continue
        
        try:
            kkma_nouns = kkma.nouns(sentence)
            nouns = [noun for noun in kkma_nouns if len(noun) >= 2]
            comment_nouns.extend(nouns)
        except Exception as e:
            print(e)
            print(sentence)
            continue

    print(f"{webtoon_no}: comment_nouns done")
    
    noun_counter = Counter(comment_nouns)

    wc = WordCloud(random_state = 123, font_path='/usr/share/fonts/NanumFont/NanumGothic.otf', width = 400, height = 400, background_color = 'white')

    img_wordcloud = wc.generate_from_frequencies(noun_counter)

    plt.figure(figsize = (10, 10)) # 크기 지정하기
    plt.axis('off') # 축 없애기
    plt.imshow(img_wordcloud)
    plt.savefig(f"wordclouds/wordcloud_{webtoon_no}.png")
    
    print(f"{webtoon_no}: save wordcloud done")